In [10]:
import os 
import pandas as pd
import numpy as np

import os
import pandas as pd
from collections import defaultdict, Counter

FOLDER1 = ["293K/", "293K/", "313K/"]
FOLDER2 = ["UPTAKE", "TIME"]

# ───────────────────────────────
# defaultdict를 사용해 중첩 딕셔너리 자동 생성
# ───────────────────────────────
def nested_dict():
    return defaultdict(nested_dict)

DICT = nested_dict()
DF_DICT = nested_dict()

# ───────────────────────────────
# 파일 탐색 및 딕셔너리 저장
# ───────────────────────────────
for F1 in FOLDER1:
    for F2 in FOLDER2:
        path = os.path.join(F1, F2)
        if not os.path.exists(path):
            continue

        for fname in os.listdir(path):
            file_path = os.path.join(path, fname)
            if not fname.endswith(".csv"):
                continue

            DF = pd.read_csv(file_path)
            DICT[F1.rstrip("/")][F2][fname] = len(DF)
            DF_DICT[F1.rstrip("/")][F2][fname] = DF

# ───────────────────────────────
# 계층 구조 예시 출력
# ───────────────────────────────
for T in DICT:
    for typ in DICT[T]:
        print(f"{T} → {typ}: {len(DICT[T][typ])} files")

# ───────────────────────────────
# 전체 카운트 요약
# ───────────────────────────────
all_lengths = [len_df for temp in DICT.values()
                        for typ in temp.values()
                        for len_df in typ.values()]

print("\n", Counter(all_lengths))


## Uptake데이터 합치기
import pandas as pd
from functools import reduce


# ───────────────────────────────
# 파일 경로 지정
# ───────────────────────────────
T2 = 293
T = str(T2) + "K"
PATHS = {
    "HENRY": f"./{T}/UPTAKE/{T2}_HENRY.csv",
    "0.01": f"./{T}/UPTAKE/{T2}_0.01.csv",
    "0.05": f"./{T}/UPTAKE/{T2}_0.05.csv",
    "0.1":  f"./{T}/UPTAKE/{T2}_0.1.csv",
    "0.5":  f"./{T}/UPTAKE/{T2}_0.5.csv",
    "1":    f"./{T}/UPTAKE/{T2}_1.csv",
    "5":    f"./{T}/UPTAKE/{T2}_5.csv",
    "15":   f"./{T}/UPTAKE/{T2}_15.csv"
}

# ───────────────────────────────
# CSV 읽고, HENRY는 henry_coeff / 나머지는 abs_mol_per_kg_framework 사용
# ───────────────────────────────
dfs = []
for label, path in PATHS.items():
    df = pd.read_csv(path)
    
    if label == "HENRY":
        if "henry_coeff" not in df.columns:
            raise KeyError(f"'{path}' 파일에서 henry_coeff 컬럼을 찾을 수 없습니다.")
        value_col = "henry_coeff"
    else:
        if "abs_mol_per_kg_framework" not in df.columns:
            raise KeyError(f"'{path}' 파일에서 abs_mol_per_kg_framework 컬럼을 찾을 수 없습니다.")
        value_col = "abs_mol_per_kg_framework"
    
    df = df[["name", value_col]].rename(columns={value_col: label})
    dfs.append(df)

# ───────────────────────────────
# name 기준 병합
# ───────────────────────────────
merged_df = reduce(lambda left, right: pd.merge(left, right, on="name", how="outer"), dfs)

# ───────────────────────────────
# 열 순서 정렬 및 결과 확인
# ───────────────────────────────
cols_order = ["name"] + list(PATHS.keys())
merged_df = merged_df[cols_order]

print(merged_df.head())

# ───────────────────────────────
# 저장
# ───────────────────────────────
merged_df.to_csv(f"./{T}_uptake_pivot.csv", index=False)
print("\n✅ 병합 ")
import pandas as pd
import numpy as np

def check_monotonic_ratio(path, verbose=True):
    """
    단조 증가하지 않는 MOF 비율 계산 엔진
    
    Parameters
    ----------
    path : str
        uptake pivot CSV 경로
    verbose : bool
        True면 통계 출력
    
    Returns
    -------
    dict
        {'file': ..., 'n_total': ..., 'n_drop_nan': ..., 'n_valid': ...,
         'n_non_mono': ..., 'ratio_non_mono': ...}
    """
    df = pd.read_csv(path)
    df = df.set_index('name')

    # 열 순서 자동 인식 (HENRY 제외 후 숫자순 정렬)
    cols = [c for c in df.columns if c != "name"]
    try:
        cols_sorted = ["HENRY"] + sorted([c for c in cols if c != "HENRY"], key=lambda x: float(x))
    except:
        cols_sorted = cols

    df = df[cols_sorted]

    n_total = len(df)

    # NaN 있는 행 제거
    df_no_nan = df.dropna(subset=cols_sorted)
    n_drop_nan = n_total - len(df_no_nan)

    # 단조성 검사
    non_mono = []
    for idx, row in df_no_nan.iterrows():
        values = row.values
        if not np.all(np.diff(values) >= 0):  # 단조 증가 실패
            non_mono.append(idx)

    n_non_mono = len(non_mono)
    n_valid = len(df_no_nan)
    ratio = n_non_mono / n_valid if n_valid > 0 else np.nan

    result = {
        'file': path,
        'n_total': n_total,
        'n_drop_nan': n_drop_nan,
        'n_valid': n_valid,
        'n_non_mono': n_non_mono,
        'ratio_non_mono': ratio
    }

    if verbose:
        print(f"📂 {path}")
        print(f"  총 MOF 개수       : {n_total}")
        print(f"  NaN 제거된 개수   : {n_drop_nan}")
        print(f"  유효 MOF 개수     : {n_valid}")
        print(f"  비단조 MOF 개수   : {n_non_mono}")
        print(f"  비단조 비율       : {ratio:.3f}")
        print("-" * 50)

    return result
def analyze_all_temperatures(paths):
    results = [check_monotonic_ratio(p) for p in paths]
    return pd.DataFrame(results)
PATHS = [
    "./273K_uptake_pivot.csv",
    "./293K_uptake_pivot.csv",
    "./313K_uptake_pivot.csv",
]
summary = analyze_all_temperatures(PATHS)
print(summary)




293K → UPTAKE: 8 files
293K → TIME: 8 files
313K → UPTAKE: 8 files
313K → TIME: 8 files

 Counter({7766: 16, 8267: 10, 7768: 5, 7668: 1})
                                    name     HENRY      0.01      0.05  \
0  1499489-acs.cgd.6b01265_1499490_clean  0.000007  0.007308  0.037157   
1                           ABAYIO_clean  0.000005  0.004859  0.024731   
2                           ABAYOU_clean  0.000005  0.005039  0.025003   
3                           ABESUX_clean  0.000001  0.001308  0.006426   
4                           ABETAE_clean  0.000001  0.001280  0.006447   

        0.1       0.5         1         5        15  
0  0.073651  0.362091  0.695022  2.768486  5.585666  
1  0.047941  0.225723  0.427736  1.658196  3.672871  
2  0.050040  0.231845  0.438342  1.706788  3.721950  
3  0.013009  0.063890  0.123668  0.536088  1.178772  
4  0.012686  0.063002  0.124638  0.532293  1.186673  

✅ 병합 
📂 ./273K_uptake_pivot.csv
  총 MOF 개수       : 8267
  NaN 제거된 개수   : 501
  유효 MOF 개수    

In [11]:
## 단조증가 깨진 것 식별
import pandas as pd
import numpy as np

def extract_nonmonotonic_mofs(path, save_csv=True):
    """
    단조 증가가 깨진 MOF만 추출하는 엔진 함수
    
    Parameters
    ----------
    path : str
        uptake pivot CSV 경로
    save_csv : bool, optional
        True면 "_nonmonotonic.csv" 파일로 저장
    
    Returns
    -------
    nonmono_df : pd.DataFrame
        단조 위반 MOF들만 담은 DataFrame
    """
    df = pd.read_csv(path)
    df = df.set_index('name')

    # 열 순서 정리 (HENRY 제외 후 숫자순 정렬)
    cols = [c for c in df.columns if c != 'name']
    try:
        cols_sorted = ['HENRY'] + sorted([c for c in cols if c != 'HENRY'], key=lambda x: float(x))
    except:
        cols_sorted = cols

    df = df[cols_sorted]
    df_no_nan = df.dropna(subset=cols_sorted)

    # 단조성 검사
    mask_nonmono = []
    for _, row in df_no_nan.iterrows():
        values = row.values
        mask_nonmono.append(not np.all(np.diff(values) >= 0))
    nonmono_df = df_no_nan[mask_nonmono]

    if save_csv:
        out_path = path.replace(".csv", "_nonmonotonic.csv")
        nonmono_df.to_csv(out_path)
        print(f"✅ {len(nonmono_df)}개 MOF 저장 완료 → {out_path}")

    return nonmono_df


# ───────────────────────────────
# 여러 온도 파일에 대해 자동 처리
# ───────────────────────────────
def extract_all_nonmonotonic(paths):
    results = {}
    for p in paths:
        results[p] = extract_nonmonotonic_mofs(p, save_csv=False)
    return results


PATHS = [
    "./273K_uptake_pivot.csv",
    "./293K_uptake_pivot.csv",
    "./313K_uptake_pivot.csv"
]
all_nonmono = extract_all_nonmonotonic(PATHS)
all_nonmono.keys()
all_nonmono["./273K_uptake_pivot.csv"]
all_nonmono["./293K_uptake_pivot.csv"]
all_nonmono["./313K_uptake_pivot.csv"]

,HENRY,0.01,0.05,0.1,0.5,1,5,15
name,,,,,,,,


In [12]:
import pandas as pd
import re
import os
from functools import reduce

def merge_time_data_horizontal(T2: int):
    """
    TIME CSV들을 MOF 기준으로 병합해
    HENRY~15bar까지 열로 갖는 horizontal pivot 생성

    예: merge_time_data_horizontal(293) → ./293K_time_horizontal.csv
    """

    # ───────────────────────────────
    # 기본 설정
    # ───────────────────────────────
    T = f"{T2}K"
    base_path = f"./{T}/TIME"

    PATHS = {
        "HENRY": f"{base_path}/{T2}_HENRY.csv",
        "0.01": f"{base_path}/{T2}_0.01.csv",
        "0.05": f"{base_path}/{T2}_0.05.csv",
        "0.1":  f"{base_path}/{T2}_0.1.csv",
        "0.5":  f"{base_path}/{T2}_0.5.csv",
        "1":    f"{base_path}/{T2}_1.csv",
        "5":    f"{base_path}/{T2}_5.csv",
        "15":   f"{base_path}/{T2}_15.csv"
    }

    dfs = []

    # ───────────────────────────────
    # 1️⃣ 각 압력 파일 읽기
    # ───────────────────────────────
    for label, path in PATHS.items():
        if not os.path.exists(path):
            print(f"⚠️  Skip (file not found): {path}")
            continue

        df = pd.read_csv(path)
        df.columns = [c.strip().lower().replace(" ", "_").replace("(s)", "s") for c in df.columns]

        # name 컬럼 및 time 컬럼 자동 탐지
        name_col = [c for c in df.columns if "mof" in c.lower() or "name" in c.lower()][0]
        time_col = [c for c in df.columns if "time" in c][0]

        # name_base에서 압력 부분 제거 (공통 식별자 통일)
        def clean_name(name):
            return re.sub(r"_\d+(?:\.\d+)?bar.*", "", str(name))
        df["name_base"] = df[name_col].apply(clean_name)

        # 필요 열만 남기고 rename
        df = df[["name_base", time_col]].rename(columns={time_col: label})
        dfs.append(df)

    if not dfs:
        raise RuntimeError("❌ No valid TIME CSV files found.")

    # ───────────────────────────────
    # 2️⃣ name_base 기준 병합
    # ───────────────────────────────
    merged_df = reduce(lambda left, right: pd.merge(left, right, on="name_base", how="outer"), dfs)

    # ───────────────────────────────
    # 3️⃣ 열 순서 정렬
    # ───────────────────────────────
    order = ["name_base", "HENRY", "0.01", "0.05", "0.1", "0.5", "1", "5", "15"]
    merged_df = merged_df[[c for c in order if c in merged_df.columns]]

    # ───────────────────────────────
    # 4️⃣ 저장
    # ───────────────────────────────
    save_path = f"./{T}_time_horizontal.csv"
    merged_df.to_csv(save_path, index=False)
    print(f"✅ Horizontal 병합 완료 → {save_path}")
    print(f"총 {len(merged_df)}개 MOF 변환됨")

    return merged_df


# ───────────────────────────────
# 실행 (예시)
# ───────────────────────────────
if __name__ == "__main__":
    for T2 in [273, 293, 313]:
        merge_time_data_horizontal(T2)


✅ Horizontal 병합 완료 → ./273K_time_horizontal.csv
총 8267개 MOF 변환됨
✅ Horizontal 병합 완료 → ./293K_time_horizontal.csv
총 7766개 MOF 변환됨
✅ Horizontal 병합 완료 → ./313K_time_horizontal.csv
총 8267개 MOF 변환됨


In [13]:
import pandas as pd 
import os 
KS = [273, 293, 313]
for K in KS:
    DF = pd.read_csv(f"./{K}K_merged_dataset.csv")
    DF
    with open("./04cif_list.txt", "r") as f:
        data = [x.strip() for x in f.readlines()]
    print(len(DF))
    DF = DF[DF["filename"].isin(data)]
    print(len(DF))
    DF.to_csv(f"./{K}K_merged_dataset.exclude.broken_cif.csv", index = False)

8267
7639
7766
7263
8267
7639


In [14]:
with open("./04cif_list.txt", "r") as f:
    data = [x.strip() for x in f.readlines()]
data
for K in KS:
    K273 = pd.read_csv(f"./{K}K_merged_dataset.exclude.broken_cif.csv")["filename"].to_list()
    print(len(set(data) - set(K273)))
    print(len(set(K273) - set(data)))
    with open(f"./추가GCMC_누락한건/04cif_list.{K}K.txt", 'w') as f:
        f.write('\n'.join(list(set(data) - set(K273))))

147
0
523
0
147
0


In [15]:
import pandas as pd

DF = pd.read_csv("./273K_merged_dataset.exclude.broken_cif.csv")
# 결측치가 하나라도 포함된 행만 필터링
na_rows = DF[DF.isna().any(axis=1)]
na_rows

,filename,LCD,PLD,LFPD,cm3_g,ASA_m2_cm3,ASA_m2_g,NASA_m2_cm3,NASA_m2_g,AV_VF,...,NAV_cm3_g,Has_OMS,HENRY,0.01,0.05,0.1,0.5,1,5,15
7,AFEKAX_clean,8.19182,6.99421,8.19182,0.987281,1565.630,1585.8000,78.053200,79.058700,0.6410,...,0.0,1,NaN,0.021477,0.107333,0.214545,1.026090,1.810673,NaN,NaN
28,BICDAU_clean,11.50132,6.68644,11.17198,0.652990,2386.530,3654.7800,0.000000,0.000000,0.7824,...,0.0,1,NaN,0.004437,0.022367,0.044683,0.221170,0.438653,NaN,NaN
36,BUKMUQ_clean,5.78794,5.06217,5.71305,0.851411,2895.400,3400.7100,0.000000,0.000000,0.7096,...,0.0,1,NaN,0.006188,0.030482,0.061313,0.306049,0.605241,NaN,NaN
40,c4ra13058h_c4ra13058h2_clean,5.16068,4.44688,5.16068,1.408960,1175.390,834.2260,0.000000,0.000000,0.5236,...,0.0,0,NaN,0.052613,0.250177,0.467211,1.626260,2.355717,NaN,NaN
74,cg801267m_si_001_clean,11.00208,5.91772,11.00208,0.888544,2113.710,2378.8500,0.000000,0.000000,0.7066,...,0.0,0,NaN,0.012008,0.061110,0.122621,0.602257,1.157064,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7551,ZAXQUM_charged,5.30150,3.54171,4.69168,1.409490,721.003,511.5350,0.000000,0.000000,0.4634,...,0.0,1,NaN,0.006039,0.030205,0.060211,0.270319,0.494686,NaN,NaN
7552,ZAYJOA_charged,4.04003,3.48900,4.02141,3.849050,202.678,52.6567,0.041703,0.010835,0.3916,...,0.0,0,NaN,0.000081,0.000406,0.000797,0.003978,0.007919,NaN,NaN
7560,ZEDVOV_charged,5.91854,3.45718,5.91854,1.243490,557.924,448.6740,35.061100,28.195700,0.4732,...,0.0,0,NaN,0.003055,0.015473,0.030234,0.151258,0.278276,NaN,NaN
7561,ZEDVUB_charged,5.32930,4.32637,5.19907,1.331440,777.591,584.0230,24.990800,18.769800,0.4496,...,0.0,0,NaN,0.003076,0.015879,0.031505,0.153867,0.294281,NaN,NaN


In [2]:
from MOF_GCMC_SAMPLER import GCMCSampler
import pandas as pd
from MOF_GCMC_DATALOADER import load_mof_dataset

df, meta = load_mof_dataset(
    csv_path="./273K_merged_dataset.exclude.broken_cif.csv",
    input_features=["LCD","PLD","LFPD","cm3_g","ASA_m2_cm3","ASA_m2_g","NASA_m2_cm3","NASA_m2_g","AV_VF","AV_cm3_g","NAV_cm3_g","Has_OMS"],
    lowp_features=["HENRY"],
    # pred_features=["pred_0.1", "pred_0.5", "pred_1"],   # ← 추가 예측 컬럼
    output_features=["1"]
)
sampler = GCMCSampler(
    sampler_type="qt_then_rd",
    qt_col="HENRY",
    use_log=True,
    n_bins=125,
    qt_frac=0.25,
    train_ratio=0.5,
    gamma=0.3,
    seed_base=2025,
    outdir="./plots"
)


result = sampler.fit(df)
sampler.summary(result, df=df)
train_idx = result["train_idx"]
test_idx = result["test_idx"]
df_train = df.iloc[train_idx]
df_test = df.iloc[test_idx]
from sklearn.preprocessing import StandardScaler
from MOF_GCMC_MODEL import MOFModelTrainer

# 예시: 이미 sampler로 train/test split 완료됨
X_train, X_test = df_train.drop(columns=[meta["meta_columns"][0], meta["output_features"][0]]), df_test.drop(columns=[meta["meta_columns"][0], meta["output_features"][0]])
y_train, y_test = df_train[meta["output_features"][0]], df_test[meta["output_features"][0]]

# 외부 스케일러 준비
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train.values.reshape(-1, 1))

trainer = MOFModelTrainer(
    model_type="rf",
    model_params={"n_estimators": 1000, "max_depth": None},
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    outdir="./run_rf"
)

trainer.fit(X_train, y_train)
metrics = trainer.evaluate(X_test, y_test)
fi = trainer.feature_importance(X_test, y_test)
pred_df = trainer.save_predictions(X_test, y_test)


📂 CSV 로드 완료: ./273K_merged_dataset.exclude.broken_cif.csv (7,639행)
🧹 결측치 제거: 376개 행 삭제 (7263개 남음)

📊 [요약]
입력 피처: 12개
저압 피처: 1개
예측 피처: 0개
출력 피처: 1개
유효 샘플: 7,263/7,639

🔍 [샘플 데이터]
                           filename       LCD       PLD      LFPD     cm3_g  \
0                      ACOCOM_clean  24.60642  13.03430  24.60642  0.349927   
1                      ACOCUS_clean  24.38417  13.22222  24.38417  0.346494   
2                      ACODAZ_clean  23.29182  13.11646  23.29182  0.343144   
3    acscombsci.5b00188_34903_clean   6.15808   5.85025   6.15808  2.165830   
4  acscombsci.5b00188_5013797_clean   9.69678   6.71185   9.61668  0.664961   

   ASA_m2_cm3  ASA_m2_g  NASA_m2_cm3  NASA_m2_g   AV_VF  AV_cm3_g  NAV_cm3_g  \
0     1537.41  4393.520          0.0        0.0  0.8630   2.46623        0.0   
1     1518.48  4382.420          0.0        0.0  0.8644   2.49471        0.0   
2     1530.83  4461.210          0.0        0.0  0.8618   2.51148        0.0   
3     1748.44   807.285    

2025-11-01 23:42:46,278 | INFO | [TRAIN] Model=RF | Samples=3341 | Features=13


📊 Plot saved → ./plots/sampling_hist_HENRY.png


2025-11-01 23:42:49,632 | INFO | [TIME] Training took 3.35 s
2025-11-01 23:42:49,964 | INFO | [EVAL] R2=0.9820 | MAE=0.0172 | RMSE=0.0013 | MAPE=2.80%
2025-11-01 23:42:50,629 | INFO | [SAVE] predictions → ./run_rf\predictions.csv


In [3]:
from MOF_GCMC_SAMPLER import GCMCSampler
import pandas as pd
from MOF_GCMC_DATALOADER import load_mof_dataset

df, meta = load_mof_dataset(
    csv_path="./273K_merged_dataset.exclude.broken_cif.csv",
    input_features=["LCD","PLD","LFPD","cm3_g","ASA_m2_cm3","ASA_m2_g","NASA_m2_cm3","NASA_m2_g","AV_VF","AV_cm3_g","NAV_cm3_g","Has_OMS"],
    lowp_features=["HENRY"],
    # pred_features=["pred_0.1", "pred_0.5", "pred_1"],   # ← 추가 예측 컬럼
    output_features=["1"]
)
sampler = GCMCSampler(
    sampler_type="qt_then_rd",
    qt_col="HENRY",
    use_log=True,
    n_bins=125,
    qt_frac=0.0,
    train_ratio=0.5,
    gamma=0.3,
    seed_base=2025,
    outdir="./plots"
)


result = sampler.fit(df)
sampler.summary(result, df=df)
train_idx = result["train_idx"]
test_idx = result["test_idx"]
df_train = df.iloc[train_idx]
df_test = df.iloc[test_idx]
from sklearn.preprocessing import StandardScaler
from MOF_GCMC_MODEL import MOFModelTrainer

# 예시: 이미 sampler로 train/test split 완료됨
X_train, X_test = df_train.drop(columns=[meta["meta_columns"][0], meta["output_features"][0]]), df_test.drop(columns=[meta["meta_columns"][0], meta["output_features"][0]])
y_train, y_test = df_train[meta["output_features"][0]], df_test[meta["output_features"][0]]

# 외부 스케일러 준비
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train.values.reshape(-1, 1))

trainer = MOFModelTrainer(
    model_type="rf",
    model_params={"n_estimators": 1000, "max_depth": None},
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    outdir="./run_rf"
)

trainer.fit(X_train, y_train)
metrics = trainer.evaluate(X_test, y_test)
fi = trainer.feature_importance(X_test, y_test)
pred_df = trainer.save_predictions(X_test, y_test)


📂 CSV 로드 완료: ./273K_merged_dataset.exclude.broken_cif.csv (7,639행)
🧹 결측치 제거: 376개 행 삭제 (7263개 남음)

📊 [요약]
입력 피처: 12개
저압 피처: 1개
예측 피처: 0개
출력 피처: 1개
유효 샘플: 7,263/7,639

🔍 [샘플 데이터]
                           filename       LCD       PLD      LFPD     cm3_g  \
0                      ACOCOM_clean  24.60642  13.03430  24.60642  0.349927   
1                      ACOCUS_clean  24.38417  13.22222  24.38417  0.346494   
2                      ACODAZ_clean  23.29182  13.11646  23.29182  0.343144   
3    acscombsci.5b00188_34903_clean   6.15808   5.85025   6.15808  2.165830   
4  acscombsci.5b00188_5013797_clean   9.69678   6.71185   9.61668  0.664961   

   ASA_m2_cm3  ASA_m2_g  NASA_m2_cm3  NASA_m2_g   AV_VF  AV_cm3_g  NAV_cm3_g  \
0     1537.41  4393.520          0.0        0.0  0.8630   2.46623        0.0   
1     1518.48  4382.420          0.0        0.0  0.8644   2.49471        0.0   
2     1530.83  4461.210          0.0        0.0  0.8618   2.51148        0.0   
3     1748.44   807.285    

2025-11-01 23:43:05,484 | INFO | [TRAIN] Model=RF | Samples=3632 | Features=13


📊 Plot saved → ./plots/sampling_hist_HENRY.png


2025-11-01 23:43:09,012 | INFO | [TIME] Training took 3.53 s
2025-11-01 23:43:09,298 | INFO | [EVAL] R2=0.9700 | MAE=0.0223 | RMSE=0.0044 | MAPE=3.04%
2025-11-01 23:43:10,058 | INFO | [SAVE] predictions → ./run_rf\predictions.csv


{'n_total': 7639,
 'n_valid': 7263,
 'input_features': ['LCD',
  'PLD',
  'LFPD',
  'cm3_g',
  'ASA_m2_cm3',
  'ASA_m2_g',
  'NASA_m2_cm3',
  'NASA_m2_g',
  'AV_VF',
  'AV_cm3_g',
  'NAV_cm3_g',
  'Has_OMS'],
 'lowp_features': ['HENRY'],
 'pred_features': [],
 'output_features': ['1'],
 'meta_columns': ['filename'],
 'dropna': True}